In [225]:
import numpy as np
from matplotlib import pyplot as plt


In [226]:
class CNN:
  def __init__(self,A_prev,W,b,hyper_parameters):
    
    self.A_prev            = A_prev
    self.W                 = W
    self.b                 = b
    self.hyper_parameters  = hyper_parameters
    
  

  def zero_pad(self,A_prev,pad):
        X = self.A_prev
        X_pad = np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),'constant',constant_values=(0,0))
        return X_pad

  def conv_single_step(self,a_slice_prev,W,b):
      s=a_slice_prev
      Z=np.sum(s)
  
      Z1=float(Z+b)
      return Z1   

  def conv_forward(self,A_prev,W,b,hyper_parameters):  

      A_prev = self.A_prev
    
      (m,n_H_prev,n_W_prev,n_C_prev) = np.shape(A_prev)
    
      W = self.W 
      (f,f,n_C_prev,n_C)=np.shape(W)

      hyper_parameters = self.hyper_parameters
      stride=hyper_parameters['stride']
      pad=hyper_parameters['pad']

      n_H = int((n_H_prev-f+2*pad)/stride)+1
      n_W = int((n_W_prev-f+2*pad)/stride)+1

      Z = np.zeros((m,n_H,n_W,n_C))

      A_prev_pad = self.zero_pad(A_prev,pad)

      
      for i in range(m):
        a_prev_pad = A_prev_pad[i,:,:,:]
    
        for h in range(n_H):
          vert_start = h*stride
          vert_end   = h*stride + f

          for w in range(n_W):
            horiz_start = w*stride
            horiz_end   = w*stride + f

            for c in range(n_C):
              a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]

              weights = W[:,:,:,c]
              biases  = b[:,:,:,c]

              Z[i,h,w,c] = self.conv_single_step(a_slice_prev,weights,biases)
              
          
          
        assert(Z.shape == (m,n_H,n_W,n_C))

        cache = (A_prev,W,b,hyper_parameters)

        return Z,cache

  def pool_forward(self,A_prev,hyper_parameters):   
        
        (m,n_H_prev,n_W_prev,n_C_prev) = np.shape(A_prev)
        A_prev = self.A_prev
        f = hyper_parameters["f"]
        stride = hyper_parameters["stride"] 

        n_H = int(1+(n_H_prev-f)/stride)
        n_W = int(1+(n_W_prev-f)/stride)
        n_C = n_C_prev    

        A = np.zeros((m,n_H,n_W,n_C))

        
        for i in range(m):
    
          for h in range(n_H):
            vert_start = h*stride
            vert_end   = h*stride + f

            for w in range(n_W):
              horiz_start = w*stride
              horiz_end   = w*stride + f

              for c in range(n_C):
                a_prev_slice = A_prev[i,vert_start:vert_end,horiz_start:horiz_end,c]

                A[i,h,w,c] = np.max(a_prev_slice)
        
        cache = (A_prev,hyper_parameters)
        assert(A.shape == (m,n_H,n_W,n_C))  
        return A,cache

 

In [227]:
class CNN_Backward:
  
  def __init__(self,A_prev,dZ,cache):
    
    self.A_prev      = A_prev
    self.cache       = cache
    self.dZ           = dZ
  
  def zero_pad(self,A_prev,pad):
        X = A_prev
        X_pad = np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),'constant',constant_values=(0,0))
        return X_pad

  def conv_backward(self,dZ, cache):
        dZ = self.dZ
        cache = self.cache
        (A_prev, W, b,hparameters) = cache
    
    
        (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    
        (f, f, n_C_prev, n_C) = W.shape
    
    
        stride = hparameters["stride"]
        pad = hparameters["pad"]
    
    
        (m, n_H, n_W, n_C) = dZ.shape
    
    
        dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))                           
        dW = np.zeros((f, f, n_C_prev, n_C))
        db = np.zeros((1, 1, 1, n_C))

    
        A_prev_pad = self.zero_pad(A_prev, pad)
        dA_prev_pad = self.zero_pad(dA_prev, pad)
    
        for i in range(m):                       
        
        
          a_prev_pad = A_prev_pad[i]
          da_prev_pad = dA_prev_pad[i]
        
          for h in range(n_H):                   
              for w in range(n_W):               
                  for c in range(n_C):           
                    
                    
                      vert_start = h
                      vert_end = vert_start + f
                      horiz_start = w
                      horiz_end = horiz_start + f
                    
                    
                      a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    
                      da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                      dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                      db[:,:,:,c] += dZ[i, h, w, c]
                    
        
          dA_prev[i, :, :, :] = da_prev_pad[pad:-pad, pad:-pad, :]
    
    
    
        assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))

        W -= 0.001 * dW
        b -= 0.001 * db

        W = np.mean(W) 
        b = np.mean(b) 
        
        return dA_prev,W,b  

In [228]:
class MAX_Backward:
  def __init__(self,dA,cache): 

    self.dA    = dA
    self.cache = cache  


  def create_mask_from_window(self,x):
    
    mask = x == np.max(x)
    return mask
  
  def distribute_value(self,dz, shape):
    
    (n_H, n_W) = shape
    average = dz / (n_H * n_W)
    a = np.ones(shape) * average
    return a
  
  def pool_backward(self,dA, cache, mode = "max"):
    
    (A_prev, hparameters) = self.cache
    
    stride = hparameters["stride"]
    f = hparameters["f"]
  
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape

    dA = self.dA
    m, n_H, n_W, n_C = dA.shape 
    
    dA_prev = np.zeros(A_prev.shape)
    
    for i in range(m):                       
        
        a_prev = A_prev[i]
        for h in range(n_H):                   
            for w in range(n_W):               
                for c in range(n_C):           
                    
                    vert_start = h
                    vert_end = vert_start + f
                    horiz_start = w
                    horiz_end = horiz_start + f
                    
                    
                    if mode == "max":
                       
                        a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, c]
                        
                        mask = self.create_mask_from_window(a_prev_slice)
                        
                        dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += np.multiply(mask, dA[i, h, w, c])
                        
                    elif mode == "average":
                        
                        da = dA[i, h, w, c]
                        
                        shape = (f, f)
                        
                        dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += distribute_value(da, shape)
                        
    
    
    
    assert(dA_prev.shape == A_prev.shape)
    
    return dA_prev  

In [247]:
np.random.seed(1)

input_image = np.random.randn(10,32,32,3)
W0 = np.random.randn(3,3,3,10)
b0 = np.random.randn(1,1,1,10)
W1 = np.random.randn(3,3,3,32)
b1 = np.random.randn(1,1,1,32)
W2 = np.random.randn(3,3,3,64)
b2 = np.random.randn(1,1,1,64)
W3 = np.random.randn(3,3,3,256)
b3 = np.random.randn(1,1,1,256)

hyper_parameters_conv = {"pad":1,"stride":2}
hyper_parameters_pool = {"stride":2,"f":3}

print("Input image dimensions                                        {}\n------------------------------------------------------------------------------ ".format(input_image.shape))

conv0 = CNN(input_image,W0,b0,hyper_parameters_conv)
Z0,cache_conv0 = conv0.conv_forward(input_image,W0,b0,hyper_parameters_conv)
print("Dimensions after applying conv0                               {}\n------------------------------------------------------------------------------  ".format(Z0.shape))

max0 = CNN(Z0,W0,b0,hyper_parameters_pool)
Z0_MAX,cache_max0 = max0.pool_forward(Z0,hyper_parameters_pool)
print("Dimensions after applying max0                                {}\n------------------------------------------------------------------------------  ".format(Z0_MAX.shape))

max_backprop0 = MAX_Backward(Z0_MAX,cache_max0)
dA_prev = max_backprop1.pool_backward(Z0_MAX, cache_max0, mode = "max")
print("Dimensions after applying max_backprop1                       {}\n------------------------------------------------------------------------------  ".format(dA_prev.shape))

conv_backprop0 = CNN_Backward(dA_prev,Z0,cache_conv0)
dA,W,b = conv_backprop0.conv_backward(Z0,cache_conv0)
print("Dimensions after applying conv_backprop0                      {}\n------------------------------------------------------------------------------  ".format(dA.shape))

Input image dimensions                                        (10, 32, 32, 3)
------------------------------------------------------------------------------ 
Dimensions after applying conv0                               (10, 16, 16, 10)
------------------------------------------------------------------------------  
Dimensions after applying max0                                (10, 7, 7, 10)
------------------------------------------------------------------------------  
Dimensions after applying max_backprop1                       (10, 16, 16, 10)
------------------------------------------------------------------------------  
Dimensions after applying conv_backprop0                      (10, 32, 32, 3)
------------------------------------------------------------------------------  
